In [8]:
from elevenlabs import ElevenLabs
from groq import Groq
import os

from app.models import ConversationLog, User
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

groq_client = Groq(api_key=GROQ_API_KEY)
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")

def infer(formatted_transcript):
    """
    Grade the conversation transcript using the Groq API.
    """
    grading_prompt = f"""
        You are a grading assistant for MBA students. Evaluate the following conversation transcript based on these criteria:
        1. **Critical Thinking**: Did the student demonstrate analytical depth and logical reasoning?
        2. **Communication**: Was the student's response clear, coherent, and well-structured?
        3. **Comprehension**: Did the student understand the case and respond appropriately?

        Provide:
        1. An overall summary of the student's performance.
        2. A final score (intgervalue between 0 and 100).
        3. Individual scores for each criterion (integervalue between 0 and 100).
        4. A performance summary with 3 strengths and 3 weaknesses, each with a title and description.
        5. Be strict in grading the student's performance.

        Transcript:
        {formatted_transcript}

        Return the response in JSON format with the following structure replacing the example values with your evaluation:
        {{
            "overall_summary": "Your performance was fair, demonstrating some understanding of the task but lacking in critical thinking and comprehension. Your communication skills were clear, but the response was limited in scope.",
            "final_score": 60,
            "individual_scores": {{
                "critical_thinking": 40,
                "communication": 80,
                "comprehension": 50
            }},
            "performance_summary": {{
                "strengths": [
                    {{"title": "Clear communication", "description": "Your response was easy to understand and well-structured."}}
                ],
                "weaknesses": [
                    {{"title": "Lack of critical thinking", "description": "You failed to demonstrate deep analytical thinking and logical reasoning in their response."}}
                ]
            }}
        }}

        CRITICAL INSTRUCTION:
        Report in the way  that shows you have analyzed the student's performance and are giving the student his/her feedback.
        No any additional text apart from the json object. 
        Do not add any ```json  or ```, return just the json object.
        """
    completion = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {
                "role": "user",
                "content": grading_prompt
            }
        ],
        temperature=0.5,
        max_completion_tokens=1024,
        top_p=1,
        stream=False,
        stop=None,
    )

    return completion.choices[0].message.content


def grade_conversation():
    """
    Fetch the conversation transcript, grade it, and return the structured JSON response.
    """
    conversation_id = "WBTq5eGH84lofidkfSD3"
    client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
    conversation = client.conversational_ai.get_conversation(conversation_id)
    transcript = conversation.transcript
    
    formatted_transcript = []
    for message in transcript:
        formatted_transcript.append({
            "role": message.role,
            "message": message.message
        })
    

    grading_response = infer(formatted_transcript)
    return grading_response

print(grade_conversation())

INFO:httpx:HTTP Request: GET https://api.elevenlabs.io/v1/convai/conversations/WBTq5eGH84lofidkfSD3 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{
    "overall_summary": "Your performance was below average, demonstrating limited understanding of the task and lacking in critical thinking, comprehension, and communication skills. You struggled to provide clear and concise answers to the questions, often failing to address the core issues and provide meaningful solutions.",
    "final_score": 40,
    "individual_scores": {
        "critical_thinking": 30,
        "communication": 50,
        "comprehension": 40
    },
    "performance_summary": {
        "strengths": [
            {"title": "Attempted to address questions", "description": "You attempted to respond to the questions posed, showing some willingness to engage with the topic."},
            {"title": "Recognized the importance of customer satisfaction", "description": "You acknowledged the need to improve customer satisfaction, which is a crucial aspect of business development."},
            {"title": "Suggested the use of surveys for feedback", "description": "You pr